In [ ]:
#!pip install pycaret[full]
!pip install scikit-learn==0.23.2 --user


In [7]:
import pycaret


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor # 머신러닝 알고리즘 모형 중
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv') 
test = pd.read_csv('data/test.csv')

In [3]:
#train.fillna(0,inplace = True)
#test.fillna(0,inplace = True)

# 시간별 평균값으로 na 채우기

def fill_na(df,column):
    print('column : ',column)
    hour_mean = train.groupby('hour').mean()[column]
    
    na_index_list = list( train[ train[column].isnull() ].index )
    
    for ind in na_index_list:
        df.loc[ind, column] = hour_mean[ train.loc[ind,'hour'] ]
        if np.isnan(hour_mean[ train.loc[ind,'hour'] ]):
            df.loc[ind, column] = train[column].mean()
    
    
fill_na(train,'hour_bef_temperature')
fill_na(test,'hour_bef_temperature')

fill_na(train,'hour_bef_windspeed')
fill_na(test,'hour_bef_windspeed')

fill_na(train,'hour_bef_humidity')
fill_na(test,'hour_bef_humidity')

fill_na(train,'hour_bef_windspeed')
fill_na(test,'hour_bef_windspeed')

fill_na(train,'hour_bef_visibility')
fill_na(test,'hour_bef_visibility')

fill_na(train,'hour_bef_ozone')
fill_na(test,'hour_bef_ozone')

fill_na(train,'hour_bef_pm10')
fill_na(test,'hour_bef_pm10')

fill_na(train,'hour_bef_pm2.5')
fill_na(test,'hour_bef_pm2.5')


train.fillna(0,inplace = True)
test.fillna(0,inplace = True)



column :  hour_bef_temperature
column :  hour_bef_temperature
column :  hour_bef_windspeed
column :  hour_bef_windspeed
column :  hour_bef_humidity
column :  hour_bef_humidity
column :  hour_bef_windspeed
column :  hour_bef_windspeed
column :  hour_bef_visibility
column :  hour_bef_visibility
column :  hour_bef_ozone
column :  hour_bef_ozone
column :  hour_bef_pm10
column :  hour_bef_pm10
column :  hour_bef_pm2.5
column :  hour_bef_pm2.5


In [4]:
train = train.drop(['id'],axis=1)
test = test.drop(['id'],axis=1)

In [9]:
train

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...
1454,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [24]:
from pycaret.regression import *

#데이터셋 정의
#어떤 전처리 들을 수행. 
reg_test_1 = setup(data=train,
                   target='count',
                   train_size=0.8,
                   feature_selection=True,
                   fold=5)



,Description,Value
0,session_id,1001
1,Target,count
2,Original Data,"(1459, 10)"
3,Missing Values,False
4,Numeric Features,8
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1167, 9)"


In [25]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,26.1422,1422.3976,37.6912,0.7873,0.5427,0.8876,0.6120
et,Extra Trees Regressor,26.0897,1475.6034,38.3708,0.7785,0.5027,0.8118,0.0840
rf,Random Forest Regressor,26.6097,1567.0359,39.5717,0.7642,0.5263,0.9110,0.0920
xgboost,Extreme Gradient Boosting,27.1377,1623.1135,40.2654,0.7571,0.5383,0.7991,0.2740
lightgbm,Light Gradient Boosting Machine,27.4624,1620.5074,40.2501,0.7567,0.5604,0.9545,0.0280
gbr,Gradient Boosting Regressor,28.0744,1645.8130,40.5001,0.7524,0.5560,0.9133,0.0320
lr,Linear Regression,39.7698,2825.4972,53.1194,0.5778,0.7934,1.2496,0.8880
lar,Least Angle Regression,39.7698,2825.5017,53.1195,0.5778,0.7934,1.2496,0.0080
ridge,Ridge Regression,40.0152,2844.9209,53.3134,0.5749,0.7824,1.2515,0.0080
ada,AdaBoost Regressor,44.0864,2876.9377,53.5188,0.5710,0.8788,1.9260,0.0260


In [12]:
et = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,25.1439,1337.9616,36.5782,0.8106,0.3266,0.2807
1,24.7672,1309.5385,36.1875,0.8198,0.4229,0.4193
2,24.6300,1215.2237,34.8601,0.8085,0.5337,0.7696
3,28.2039,2037.4983,45.1387,0.6994,0.6130,1.7088
4,28.1666,1615.1359,40.1888,0.7531,0.6166,0.9883
Mean,26.1823,1503.0716,38.5906,0.7783,0.5026,0.8334
Std,1.6440,298.5777,3.7195,0.0459,0.1127,0.5043


In [13]:
cb = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,26.6758,1416.2108,37.6326,0.7995,0.3669,0.3219
1,25.2245,1185.8525,34.4362,0.8369,0.4961,0.5158
2,25.2279,1188.7571,34.4784,0.8127,0.5366,0.7994
3,26.7066,2013.1414,44.8680,0.7030,0.5822,1.7166
4,28.5748,1667.3068,40.8327,0.7451,0.6601,1.1886
Mean,26.4819,1494.2537,38.4496,0.7794,0.5284,0.9085
Std,1.2346,314.2362,3.9855,0.0486,0.0974,0.4983


# 하이퍼파라미터 튜닝

In [14]:
et = create_model('et')
et = tune_model(et,
                optimize='RMSE',
                n_iter=100, 
               )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,25.6961,1459.4700,38.2030,0.7934,0.3398,0.2804
1,25.4869,1403.6300,37.4650,0.8069,0.4364,0.4531
2,24.8395,1278.0092,35.7493,0.7986,0.5446,0.8116
3,28.5677,2071.3603,45.5122,0.6944,0.6084,1.6364
4,28.4603,1733.4135,41.6343,0.7350,0.6290,1.0638
Mean,26.6101,1589.1766,39.7128,0.7657,0.5117,0.8491
Std,1.5803,283.3309,3.4747,0.0438,0.1090,0.4791


In [15]:
print(et)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                    max_depth=10, max_features=1.0, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.001,
                    min_impurity_split=None, min_samples_leaf=2,
                    min_samples_split=10, min_weight_fraction_leaf=0.0,
                    n_estimators=160, n_jobs=-1, oob_score=False,
                    random_state=2399, verbose=0, warm_start=False)


In [16]:
# finaiize

final_model = finalize_model(et)

#test set
prediction = predict_model(final_model,
                           data=test)


In [17]:
prediction

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,Label
0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0,101.259375
1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0,221.434375
2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0,50.806250
3,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0,46.462500
4,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0,40.840625
...,...,...,...,...,...,...,...,...,...,...
710,1,24.6,0.0,2.4,60.0,1745.0,0.000,0.0,0.0,65.415625
711,1,18.1,0.0,1.0,55.0,2000.0,0.000,0.0,0.0,58.256250
712,9,23.3,0.0,2.3,66.0,1789.0,0.020,17.0,15.0,113.496875
713,16,27.0,0.0,1.6,46.0,1956.0,0.032,40.0,26.0,164.118750


In [18]:
submission = pd.read_csv("./data/submission.csv")
submission

,id,count
0,0,1
1,1,1
2,2,1
3,4,1
4,5,1
...,...,...
710,2148,1
711,2149,1
712,2165,1
713,2166,1


In [19]:
submission['count']=prediction['Label']

In [20]:
submission.to_csv("베이스라인.csv",index=False)